In [1]:
SIMULATION_DATE = "2025-07-15"

from config import r,STOCKS,FILEPATH,s3,EXCHANGE
import os
def download_file():
    if not os.path.exists(f'{FILEPATH}/{EXCHANGE}/{STOCKS[0]}/{SIMULATION_DATE}.csv'):
        try:
            s3.download_file(Bucket='kite', Key=f'{EXCHANGE}/{STOCKS[0]}/{SIMULATION_DATE}.csv', Filename=f'{FILEPATH}/{EXCHANGE}/{STOCKS[0]}/{SIMULATION_DATE}.csv')
            print(f"[MAIN] Downloaded {EXCHANGE}:{STOCKS[0]} for {SIMULATION_DATE}")
        except Exception as e:
            raise Exception("file not found,choose another date.")
download_file()

from simulator import get_all_tick_data
df = get_all_tick_data(SIMULATION_DATE)

from StockAnalyser import Cumulative_Support
instance = Cumulative_Support()
import pandas as pd
for row in df.rolling(1):
    instance.parse(row.iloc[0].to_dict())


[INFO] loaded data into memory for simulation


/Users/gurusai/programming/STONKS/BackTestingEngine/app/StockAnalyser.py:103: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.ltpDf = pd.concat([self.ltpDf, pd.DataFrame(new_record)], ignore_index=True)


In [ ]:
import numpy as np

df = instance.voldiff_sell
def findPeaks(df):
    df['diff'] = df[0]-df[0].shift(1)
    ridingPeak = False
    start=0
    peaks = []
    start_index = 0
    for index in range(len(df['diff'])):
        diff = df['diff'].loc[index]
        if diff>10 and not ridingPeak:
            ridingPeak=True
            start = df[0].loc[index-1]
            start_index = index
            continue
        if diff<0 and ridingPeak:
            if df[0].loc[index]-start >40:
                print(f'peak ended at {index},began at {start_index},ltpvalue:{instance.ltpDf["ltp"].loc[index]}')
                peaks.append((index,df[0].loc[index]))
            ridingPeak=False
    return peaks
findPeaks(instance.voldiff_buy) #example

peak ended at 2522,began at 2520,ltpvalue:7291.0
peak ended at 2526,began at 2523,ltpvalue:7290.0
peak ended at 3419,began at 3417,ltpvalue:7331.0
peak ended at 3649,began at 3647,ltpvalue:7349.0
peak ended at 3959,began at 3954,ltpvalue:7361.0
peak ended at 4026,began at 4024,ltpvalue:7356.0
peak ended at 4648,began at 4647,ltpvalue:7361.0


[(2522, np.float64(94.25328827071553)),
 (2526, np.float64(139.32525547011437)),
 (3419, np.float64(64.15984881541677)),
 (3649, np.float64(108.40590455329036)),
 (3959, np.float64(173.3675730695661)),
 (4026, np.float64(161.9989311544663)),
 (4648, np.float64(130.368150271231))]

In [6]:
import pyqtgraph as pg
import pandas as pd
import numpy as np

# ======== INITIALISATION ============
app = pg.mkQApp()

win_buy = pg.GraphicsLayoutWidget(title="Buy Heatmap & Trends")
win_buy.setBackground("white")
# CREATE PLOTS 
plot_buy = win_buy.addPlot(title="Buy Side")
plot_diff = win_buy.addPlot(title="DIFF",row=1, col=0)
plot_sell = win_buy.addPlot(title="SELL",row=2, col=0)
# link x axes for plots
plot_buy.setXLink(plot_diff)
plot_diff.setXLink(plot_sell)
plot_sell.setYLink(plot_buy)
win_buy.show()
# ======== HELPER FUNCTIONS ========
def addHeatMap(plot,data):
    # Create a colormap with transparency
    """
    colors = [
        (1, (0, 255, 0, 200)), # Green with light alpha
        (0, (255, 0, 0, 200))    # Red with light alpha
    ]
        
    cmap = pg.ColorMap(pos=np.array([c[0] for c in colors]), color=np.array([c[1] for c in colors], dtype=np.ubyte))
    img = pg.ImageItem(data)
    img.setOpacity(.5)
    lut = cmap.getLookupTable(nPts=256, alpha=True)  # Enable alpha
    img.setLookupTable(lut)
    plot.addItem(img)
    return img"""
    
    lut = np.array([
        [254,   79,   92,   200],  # 0: red (0,0)
        [198, 158,   81,   200],  # 1: brown   (0,1)
        [146, 194,   79,   200],  # 2: green (1,0)
        [7,   235, 59,   200],  # 3: neon green  (1,1)
    ], dtype=np.ubyte)

    # Create the ImageItem
    img = pg.ImageItem(data)

    # Assign the LUT
    img.setLookupTable(lut)

    # No need for setOpacity()—alpha comes from LUT
    plot.addItem(img)
    return img
def getXY(data: pd.Series):
    if data is None or data.empty or data.shape == 0:
        return [], []
    if isinstance(data, pd.DataFrame):
    # Take first column automatically
        data = data.iloc[:, 0]
    x = instance.ltpDf.index.to_numpy()
    
    # Convert to numeric if data is not already float or int
    if data.dtypes not in [np.float64, np.int64]:  
        y = pd.to_numeric(data, errors='coerce').to_numpy()
    else:
        y = data.to_numpy()

    # Ensure x and y have the same length
    min_len = min(len(x), len(y))
    x, y = x[:min_len], y[:min_len]  

    # Create mask and apply it safely
    mask = ~np.isnan(y)
    mask = mask[:min_len]  # Ensure mask has the same length as x and y

    return x[mask], y[mask]  # Filter out NaN values        
def addlinePlot(plot,linewidth,data:pd.Series,name,color=None):
    x,y = getXY(data)
    if color==None:
        return plot.plot(x,y,pen=pg.mkPen( width=linewidth,name=name))
    return plot.plot(x,y,pen=pg.mkPen(color=color, width=linewidth,name=name))
def addScatterPlot(plot,size,data,color):
    x = [x[0]for x in data]
    y = instance.ltpDf['ltp'].loc[x]
    scatter = pg.ScatterPlotItem(x=x, y=y, pen=pg.mkPen(color=color), symbol='x',
                             brush=pg.mkBrush(0, 0, 255, 120), size=size)
    return plot.addItem(scatter)


# ===== CREATE ELEMENTS ========
#BUY
instance.hmap_buy = addHeatMap(plot_buy, instance.combineddf[0].to_numpy().T)
instance.line_upper_1_buy = addlinePlot(plot_buy, linewidth=4, data=instance.lowHighMaxes[0], name='Buy Uptrend', color='#097969')
instance.line_lower_1_buy = addlinePlot(plot_buy, linewidth=4, data=instance.HighlowMaxes[0], name='Buy Downtrend', color='#fbd604')
instance.line_ltp_buy = addlinePlot(plot_buy, linewidth=2, data=instance.ltpDf['ltp'], name='LTP', color=(0, 0, 255))
#SELL
instance.hmap_sell = addHeatMap(plot_sell, instance.combineddf[1].to_numpy().T)
instance.line_upper_1_sell = addlinePlot(plot_sell, linewidth=4, data=instance.lowHighMaxes[1], name='Sell Uptrend', color='#097969')
instance.line_lower_1_sell = addlinePlot(plot_sell, linewidth=4, data=instance.HighlowMaxes[1], name='Sell Downtrend', color='#fbd604')
instance.line_ltp_sell = addlinePlot(plot_sell, linewidth=2, data=instance.ltpDf['ltp'], name='LTP', color=(0, 0, 255))
#DIFF
instance.vol_diff_50 = addlinePlot(plot_diff, linewidth=4, data=instance.voldiff_buy, name='VolDiff_buy', color='#fdd750')
instance.vol_diff_20 = addlinePlot(plot_diff, linewidth=4, data=instance.voldiff_sell, name='VolDiff_sell', color='#097969')
instance.scatter_buy = addScatterPlot(plot_buy,15,findPeaks(instance.voldiff_buy),color="#0BF05BFF")
instance.scatter_sell = addScatterPlot(plot_sell,15,findPeaks(instance.voldiff_sell),color="#F00B0BFF")
# ======== UPDATE VALUES ========
#heatmap
instance.hmap_buy.setImage(instance.combineddf[0].to_numpy().T, autoLevels=False)
instance.hmap_sell.setImage(instance.combineddf[1].to_numpy().T, autoLevels=False)
min_ltp = instance.ltpDf['ltp'].min()
n = len(instance.ltpDf)
m = len(instance.aggDf)
instance.hmap_buy.setRect(0, min_ltp, n, m)
instance.hmap_sell.setRect(0, min_ltp, n, m)

#ltp
x,y_ltp = getXY(instance.ltpDf['ltp'])
instance.line_ltp_buy.setData(x,y_ltp)
instance.line_ltp_sell.setData(x,y_ltp)
#voldiff buy
x,y = getXY(instance.voldiff_buy[0])
instance.vol_diff_50.setData(x, y)
#voldiff sell
x,y = getXY(instance.voldiff_sell[0])
instance.vol_diff_20.setData(x, y)

# ======= SETUP ==========
plot_diff.showGrid(x=True, y=True)
plot_buy.showGrid(x=True,y=True)
plot_sell.showGrid(x=True,y=True)
app.exec()


peak ended at 2522,began at 2520,ltpvalue:7291.0
peak ended at 2526,began at 2523,ltpvalue:7290.0
peak ended at 3419,began at 3417,ltpvalue:7331.0
peak ended at 3649,began at 3647,ltpvalue:7349.0
peak ended at 3959,began at 3954,ltpvalue:7361.0
peak ended at 4026,began at 4024,ltpvalue:7356.0
peak ended at 4648,began at 4647,ltpvalue:7361.0
peak ended at 11,began at 7,ltpvalue:7230.0
peak ended at 2255,began at 2252,ltpvalue:7287.0
peak ended at 2404,began at 2403,ltpvalue:7287.0
peak ended at 2536,began at 2533,ltpvalue:7293.0
peak ended at 2574,began at 2572,ltpvalue:7290.0
peak ended at 3133,began at 3132,ltpvalue:7310.0
peak ended at 3692,began at 3691,ltpvalue:7351.0
peak ended at 4001,began at 3992,ltpvalue:7357.0
peak ended at 4080,began at 4079,ltpvalue:7356.0
peak ended at 5067,began at 5065,ltpvalue:7369.0


0

# Premium Absorption

In [18]:
instance.combineddf[0].iloc[:,25].loc[7374]

np.float64(nan)

In [11]:
instance.combineddf[0].columns[6]

'buy-vol'